In [1]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

from dotenv import load_dotenv
load_dotenv()

embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

loader = TextLoader("./data/acme_bank_faq.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(documents)
print("- loaded bank docs")
vector_store = FAISS.from_documents(splits, embeddings)
print("- created bank index")

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(
        search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    verbose=False
)
result = qa_chain.invoke({
    "question": "How do I unblock my credit card?",
    "chat_history": []
})
print(result["answer"])

- loaded bank docs
- created bank index


{'question': 'How do I unblock my credit card?',
 'chat_history': [],
 'answer': "To unblock your credit card, follow these steps:\n\n1. **Access Online Banking**: Log in securely to your online banking account.\n2. **Navigate to the Accounts Overview**: Click on the credit card that you need to unblock.\n3. **Under 'Manage Your Card,' Select 'Block/Unblock'**: Follow the pop-up instructions for confirmation.\n4. **Proceed with Changes**: Confirm to unblock the card immediately.\n\nIf online banking isn’t available, you can visit a branch or contact support for assistance.",
 'source_documents': [Document(metadata={'source': './data/acme_bank_faq.txt'}, page_content="Visit a branch or contact support if online banking isn’t available.\n\n**Blocking or Unblocking a Credit Card**\n\nTo block/unblock a credit card:\n\n1. **Access Online Banking**: Log in securely.\n2. **Navigate to the Accounts Overview**: Click on the credit card needing modifications.\n3. **Under 'Manage Your Card,' Sel